In [ ]:
# Premonitor: model_blueprints.py
# This file serves as an architecture library, containing the functions
# that define the structure of our core AI models.
# This code is used during the training phase on a powerful PC before
# the final, trained models are deployed to the Raspberry Pi.

import tensorflow as tf
from tensorflow.keras import layers, models, applications
import config

def get_thermal_anomaly_model(input_shape=config.THERMAL_MODEL_INPUT_SHAPE):
    """
    Creates the SimSiam model structure for thermal anomaly detection.
    This is based on the methodology from the Goyal & Rajapakse paper,
    using a pre-trained XceptionNet backbone for powerful feature extraction.

    Returns:
        A tuple containing the siamese_model, encoder, and predictor.
    """
    backbone = applications.Xception(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    backbone.trainable = True

    encoder_input = layers.Input(input_shape, name="encoder_input")
    x = backbone(encoder_input, training=True)
    x = layers.GlobalAveragePooling2D(name="avg_pool")(x)
    x = layers.Dense(2048, activation='relu', name="encoder_dense_1")(x)
    encoder_output = layers.BatchNormalization(name="encoder_bn")(x)
    encoder = models.Model(encoder_input, encoder_output, name="encoder")

    predictor = models.Sequential([
        layers.Input(shape=(2048,), name="predictor_input"),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(2048)
    ], name="predictor")

    input_a = layers.Input(input_shape, name="view_a")
    input_b = layers.Input(input_shape, name="view_b")
    z_a = encoder(input_a)
    z_b = encoder(input_b)
    p_a = predictor(z_a)
    p_b = predictor(z_b)
    siamese_model = models.Model([input_a, input_b], [p_a, z_a, p_b, z_b], name="siamese_model")

    print("--- Blueprint created: Thermal Anomaly Model (SimSiam with XceptionNet) ---")
    return siamese_model, encoder, predictor

def get_acoustic_anomaly_model(input_shape=config.ACOUSTIC_MODEL_INPUT_SHAPE):
    """
    Creates a basic CNN for classifying spectrograms.
    TODO: For V2.0, upgrade this to a GhostNetV2 implementation for better efficiency.
    """
    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ], name="acoustic_cnn")

    print("\n--- Blueprint created: Acoustic Anomaly Model (Spectrogram CNN) ---")
    return model

def get_lstm_autoencoder_model(timesteps, n_features):
    """
    Creates an LSTM Autoencoder model for sequential time-series data.
    Based on the architecture from the Reis & Serôdio paper.
    """
    model = models.Sequential([
        layers.Input(shape=(timesteps, n_features)),
        layers.LSTM(32, activation='relu', return_sequences=True),
        layers.LSTM(16, activation='relu', return_sequences=False),
        layers.RepeatVector(timesteps),
        layers.LSTM(16, activation='relu', return_sequences=True),
        layers.LSTM(32, activation='relu', return_sequences=True),
        layers.TimeDistributed(layers.Dense(n_features))
    ], name="lstm_autoencoder")

    print("\n--- Blueprint created: Time-Series LSTM Autoencoder ---")
    return model